In [ ]:
!pip uninstall -y tensorflow
!pip install transformers datasets tokenizers

Found existing installation: tensorflow 2.9.2
Uninstalling tensorflow-2.9.2:
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 744.5 kB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into accou

In [ ]:
from tokenizers import ByteLevelBPETokenizer

path = '/content/drive/MyDrive/Datasets/avestan_corpus.csv'

tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files=path, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

tokenizer.save_model("/content/drive/MyDrive/AvestaBERT/")

['/content/drive/MyDrive/AvestaBERT/vocab.json',
 '/content/drive/MyDrive/AvestaBERT/merges.txt']

In [ ]:
from transformers import RobertaTokenizerFast

#tokenizer files must be named as vocab.json and merges.txt
tokenizer = RobertaTokenizerFast.from_pretrained('/content/drive/MyDrive/AvestaBERT/', model_max_length=512)

In [ ]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-large")

Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

XLMRobertaForMaskedLM(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0): XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True

In [ ]:
#model.resize_token_embeddings(len(tokenizer))
#model.init_weights()
new_embeddings = torch.nn.Embedding(len(tokenizer), 1024)
model.set_input_embeddings(new_embeddings)

In [ ]:
from datasets import load_dataset

dataset_raw = load_dataset("csv", data_files='/content/drive/MyDrive/Datasets/avestan_corpus.csv')

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from datasets import DatasetDict

def tokenization(raw):
  result = raw
  result["input_ids"] = tokenizer.encode(raw["Verse"], truncation=True)
  return result

dataset = dataset_raw["train"].map(tokenization)
#dataset = DatasetDict({"train": tokenized_train})

  0%|          | 0/73580 [00:00<?, ?ex/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/AvestaBERT/",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    gradient_accumulation_steps=32,
)

trainer=Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
%%time
trainer.train()

The following columns in the training set don't have a corresponding argument in `XLMRobertaForMaskedLM.forward` and have been ignored: Verse, Source, Address, Book. If Verse, Source, Address, Book are not expected by `XLMRobertaForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 73580
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 32
  Total optimization steps = 3447
  Number of trainable parameters = 410438432


Step,Training Loss
500,18.205300
1000,27.159900
1500,20.815600
2000,22.750000


In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

In [ ]:
fill_mask("ustānazastō. <mask>. manyə̄uš.")

[{'score': 0.35884353518486023,
  'token': 18,
  'token_str': '.',
  'sequence': 'ustānazastō... manyə̄uš.'},
 {'score': 0.029511934146285057,
  'token': 290,
  'token_str': '̌',
  'sequence': 'ustānazastō.̌. manyə̄uš.'},
 {'score': 0.02216900885105133,
  'token': 303,
  'token_str': '̰.',
  'sequence': 'ustānazastō.̰.. manyə̄uš.'},
 {'score': 0.017115630209445953,
  'token': 300,
  'token_str': ' aṣ',
  'sequence': 'ustānazastō. aṣ. manyə̄uš.'},
 {'score': 0.013160294853150845,
  'token': 345,
  'token_str': '̊.',
  'sequence': 'ustānazastō.̊.. manyə̄uš.'}]